Objective:
    - scrape data from the personal finance reddit thread

We will use PRAW for this (python wrapper for the Reddit API)

In [ ]:
#  For reddit API
!pip install praw

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import praw
import yaml
cred = yaml.load(open('./auth/auth.yaml'))

In [21]:
cred

{'auth': {'client': 'gGFI0Pf5eKyIZg',
  'name': 'RedHomePF',
  'pass': 'jFeFwCvg3Zmg0XmYE9CeK4vEU0w'}}

In [22]:
reddit = praw.Reddit(client_id=cred['auth']['client'], client_secret=cred['auth']['pass'], user_agent=cred['auth']['name'])

In [23]:
reddit

In [25]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit('MachineLearning').hot(limit=20)
for post in hot_posts:
    print(post.title)

Regarding beginner's guides
[D] Machine Learning - WAYR (What Are You Reading) - Week 68
[D] Why is PyTorch as fast as (and sometimes faster than) TensorFlow?
[R] AdvHat: Real-world adversarial attack on ArcFace Face ID system
[P] Introducing Deepkit - the first collaborative desktop app for deep learning experiments. Experiment tracking, model debugging, infrastructure management.
[R] [1906.03671] Deep Batch Active Learning by Diverse, Uncertain Gradient Lower Bounds
[N] [CFP] 3rd edition of Emergent Communication workshop at NeurIPS'19 (EmeCom)
[D] Machine Learning - WAYR (What Are You Reading) - Week 69
[R] FacebookAI releases Adaptive attention span and All-attention layer to reduce decrease computation time / memory footprint
[D] How can I reduce the difference between real and predicted stock price?
[D] Classification of irregular time series
[D] Interview with two senior data scientists at Microsoft about deep learning
[R] YOLACT: Real-time Instance Segmentation
ADMM for Efficie

In [36]:
# get hottest posts from all subreddits
hot_posts = reddit.subreddit('personalfinance').hot(limit=10)
for post in hot_posts:
    print(post.title,len(post.comments.list()))

New to /r/personalfinance? Have questions? Read this first! 0
Weekday Help and Victory Thread for the week of August 26, 2019 0
Start thinking of "saving money" as "making money" instead. 165
Attitude about finances matters a lot. Don't be a "victim" of your budget. 310
How to stop living paycheck to paycheck? 34
Stocks don't have to be scary (if you hold for long enough) 43
21,151 CC Debt 22,000 Student Loan Debt just want advice 3
Where to put future down payment money - savings account or principal repayment in current home? 2
Need life advice on just about everything. 5
Fixing all my debt slowly vs. using 401K funds 6


In [41]:
posts = []
ml_subreddit = reddit.subreddit('personalfinance')
for post in ml_subreddit.hot(limit=40):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])


In [50]:
posts.sort_values('num_comments',ascending=False).head()

,title,score,id,subreddit,url,num_comments,body,created
3,Attitude about finances matters a lot. Don't b...,3953,cv6ffb,personalfinance,https://www.reddit.com/r/personalfinance/comme...,328,Ive learned this recently and it really has ch...,1.566754e+09
2,"Start thinking of ""saving money"" as ""making mo...",5970,cvbu8u,personalfinance,https://www.reddit.com/r/personalfinance/comme...,186,"I'm no expert, but this simple change in thoug...",1.566785e+09
21,Someone is trying to gain access to multiple a...,310,cv4o0q,personalfinance,https://www.reddit.com/r/personalfinance/comme...,78,Sorry if this is the wrong place. I don’t know...,1.566740e+09
5,Stocks don't have to be scary (if you hold for...,138,cvabx2,personalfinance,https://www.reddit.com/r/personalfinance/comme...,46,Hi Personal Finance!\n\nI held back from index...,1.566778e+09
4,How to stop living paycheck to paycheck?,6,cvkwmx,personalfinance,https://www.reddit.com/r/personalfinance/comme...,35,"So, I'm currently unemployed but looking for a...",1.566834e+09


In [43]:
# Obtain comments on the post about "attitues about finance"
submission = reddit.submission(id="cv6ffb")

'True'

In [79]:
submission.comments.replace_more(limit=0)
attitude_cmt = []
for comment in submission.comments.list():
    c_ed = 'Unedited' if comment.edited==False else 'Edited'
    attitude_cmt.append([comment.id,comment.body,comment.ups,comment.downs,comment.depth,comment.score,c_ed,comment.created])
    #print(comment.body,comment.ups)
df_att = pd.DataFrame(attitude_cmt,columns=['id', 'body', 'ups', 'downs', 'depth', 'score', 'edited', 'created'])

In [81]:
df_att.head()

,id,body,ups,downs,depth,score,edited,created
0,ey27v7c,Interesting perspective on considering afforda...,133,0,0,133,Unedited,1.566758e+09
1,ey2e57v,"This is a good point. In college, I worked a m...",784,0,0,784,Unedited,1.566765e+09
2,ey26lfn,This also is a good way to perceive your money...,611,0,0,611,Unedited,1.566756e+09
3,ey2kaoe,I see a lot of posts on here where people regr...,39,0,0,39,Unedited,1.566771e+09
4,ey2agnc,My father is a retired auto mechanic. How much...,170,0,0,170,Edited,1.566761e+09


In [82]:
df_att.to_pickle('./data/attitudes_to_finance.pkl')